In [1]:
import sys
!{sys.executable} -m pip install wand

In [2]:
import sys
!{sys.executable} -m pip install opencv-python

In [1]:
from itertools import chain
from wand.color import Color 
from wand.image import Image as WandImage
from wand.display import display
import cv2
import numpy as np
from PIL import Image as PILImage
import os

In [11]:
os.makedirs("inputs/", exist_ok = True)
os.makedirs("outputs/", exist_ok = True)

In [3]:
def smooth_image(original_frame, cropped_face, face_dimension, face_center, output_path):
  print(cropped_face.shape, type(cropped_face),  cropped_face.dtype)
  cv2.imshow("cropped face", cropped_face)
  print("original_frame")
  dst = cv2.imread(original_frame)
  print(dst.shape, type(dst), dst.dtype)
  ## Now, we create a mask for the face
  src_mask = np.zeros(cropped_face.shape, cropped_face.dtype)
  face_w, face_h = face_dimension
  poly = np.array([[0,0], [face_w, 0], [face_w, face_h], [0, face_h]], np.int32)
  cv2.fillPoly(src_mask, [poly], (255, 255, 255))

  ## Now we are ready to call seamlessClone 
  output = cv2.seamlessClone(cropped_face, dst, src_mask, face_center, cv2.NORMAL_CLONE)
  cv2.imwrite(output_path, output)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
  print("Final output: ")

In [4]:
face_dimension = (180, 300) #w, h

In [5]:
rectangle_face_coordinates = (   
    ( 705.87203406,   34.74937345),
    (678.39937345,  534.02796594),
    (1177.67796594,  561.50062655),
    (1205.15062655,   62.22203406)
)

In [6]:
'''
cover:  small picture (just the face)
template: big picture (al the frame)
'''
def transform(template_path, cover_path, rectangle_face_coordinates, face_dimension, output_path):
    with WandImage(filename=template_path) as template:
        cover_array = cv2.imread(cover_path) #, cv2.IMREAD_UNCHANGED)
        cover_array_resized = cv2.resize(cover_array, (template.size[1], template.size[0]))      
        with WandImage.from_array(cover_array_resized, channel_map="BGR") as cover:
            w, h = cover.size
            print(f"Size of original cover: {(w, h)}")
            print(f"Size of cover resized: {(cover.size)}")
            print(f"Size of template: {(template.size)}")
            cover.virtual_pixel = 'transparent'
            source_points = (
                  (0, 0),
                  (0, h),
                  (w, h),
                  (w, 0)
            )
            order = chain.from_iterable(zip(source_points, rectangle_face_coordinates))
            arguments = list(chain.from_iterable(order))
            print(f"arguments: {arguments}")
            cover.distort('perspective', arguments)
            # cover.save(filename='./intermediate_outputs/face_in_perspective.jpg')
            ###################################
            ### Here, we crop just the face ###
            x_center = int((rectangle_face_coordinates[0][0]+ rectangle_face_coordinates[2][0])/2)
            y_center = int((rectangle_face_coordinates[0][1]+ rectangle_face_coordinates[2][1])/2)
            face_w, face_h = face_dimension
            print(f"face_dimension: {face_dimension}")
            left = x_center - (face_w // 2)
            top = y_center - (face_h // 2)
            print(f"left: {left}, top: {top}")
            cover.crop(left= left, top= top, width=face_w, height=face_h)
            ##################################
            pixels = np.array(cover.export_pixels(channel_map= 'BGR', storage='char'), dtype="uint8").reshape((face_h, face_w, 3))
            smooth_image(original_frame, pixels, face_dimension, (x_center, y_center), output_path)

In [7]:
original_frame = "./inputs/original_frame.png"
aligned_sad_image = "./inputs/newswoman_super_happy_face.jpg"
aligned_sad_image = "./inputs/newswoman_sad_face.jpg"
output_path = "./outputs/newswoman_output_1.jpg"

In [9]:
transform(original_frame, aligned_sad_image, rectangle_face_coordinates, face_dimension, output_path)

Size of original cover: (1080, 1920)
Size of cover resized: (1080, 1920)
Size of template: (1920, 1080)
arguments: [0, 0, 705.87203406, 34.74937345, 0, 1920, 678.39937345, 534.02796594, 1080, 1920, 1177.67796594, 561.50062655, 1080, 0, 1205.15062655, 62.22203406]
face_dimension: (180, 300)
left: 851, top: 148
(300, 180, 3) <class 'numpy.ndarray'> uint8
original_frame
(1080, 1920, 3) <class 'numpy.ndarray'> uint8
Final output: 


In [12]:
# with WandImage(filename="./intermediate_outputs/news_resized_test.jpg") as function_resized, WandImage(filename="./inputs/newswoman_super_happy_face.jpg") as self_resized :
#     print(self_resized.size)
#     self_resized.resize(1920, 1080)
#     self_resized_pixels = self_resized.export_pixels(storage="char")
#     print(f"num pixels for self_resized: {len(self_resized_pixels)}")
#     print(self_resized_pixels[:20])
# #     print(self_resized.colors)
#     print(self_resized.depth)
    
#     print()
    
#     print(function_resized.size)
#     function_resized_pixels = function_resized.export_pixels(storage="char")
#     print(f"num pixels for function_resized: {len(function_resized_pixels)}")
#     print(function_resized_pixels[:20])
# #     print(function_resized.colors)
#     print(function_resized.depth)
    
#     print()
    
#     print(max(abs(self_resized_pixels[i]-function_resized_pixels[i]) for i in range(len(self_resized_pixels))))


    

In [33]:
# print(f"resolution: {cropped_resolution}")
# cropped_face = np.array(cropped_face, dtype="uint8").reshape(300, 180, 3)
# cv2.imwrite("cropped_test.jpg", cropped_face)
# print(cropped_face[0][:10])
# print(cropped_face.shape, cropped_face.dtype)

In [34]:
# pre = cv2.imread("./intermediate_outputs/cropped_face.jpg")
# print(pre[0][:10])
# print(pre.shape)

In [35]:
# cropped_face_path = "./intermediate_outputs/cropped_face.jpg"
# # cropped_face =
# smooth_image(original_frame, cropped_face , face_dimension, center)